In [48]:
import pandas as pd
import os
from tqdm import tqdm
from utils import avg, evidence_to_mask

def to_data_df(df, data_dir):
    data_df = []
    columns = ['text', 'classification', 'rationale']
    for i in tqdm(range(len(df))):
        df_row = df.loc[i]
        doc_ids = df_row['docids']
        query = df_row['query']
        evidence_list = df_row['evidences']
        if evidence_list:
            evidence_list = [x for xx in evidence_list for x in xx]
        classification = df_row['classification']
        
        text = ''
        for doc in doc_ids:
            file = f'{data_dir}/docs/{doc}'
            if os.path.isfile(file):
                f = open(file, 'r', encoding="utf-8") 
                for line in f.readlines():
                    text += line.rstrip() + ' '
            else:
                print("???")
                print(file)
                quit()
        
        tokens = text.split()
        rationale_mask = evidence_to_mask(tokens, evidence_list)
        
        # joining text and query with [SEP]
        QA = f"{query} [SEP] {text}"
        rationale_mask = [1]*(len(query.split())+1) + rationale_mask
    
        data_df.append([QA, classification, rationale_mask])
    data_df = pd.DataFrame(data_df, columns=columns)
    return data_df
    
    data_df_shuffled=data_df.sample(frac=1).reset_index(drop=True)
    return data_df_shuffled

In [45]:
dataset = "boolq"

In [46]:
data_dir = f'../data/{dataset}'
train = pd.read_json(f'{data_dir}/train.jsonl', lines=True)
test = pd.read_json(f'{data_dir}/test.jsonl', lines=True)
val = pd.read_json(f'{data_dir}/val.jsonl', lines=True)

In [42]:
def class_distr(df):
    classifications = list(df['classification'])
    class_dict = {}
    for c in classifications:
        if c not in class_dict.keys():
            class_dict[c] = 1
        else:
            class_dict[c] += 1

    counts = list(class_dict.values())
    distribution = [x/sum(counts) for x in counts]

    return class_dict, distribution
class_distr(train)
# test.info()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6363 entries, 0 to 6362
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   annotation_id   6363 non-null   int64  
 1   classification  6363 non-null   object 
 2   docids          6363 non-null   object 
 3   evidences       6363 non-null   object 
 4   query           6363 non-null   object 
 5   query_type      0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 298.4+ KB


In [49]:
train_data_df = to_data_df(train, data_dir)
train_data_df.to_csv(f"others/{dataset}/train.csv",index_label="id")
test_data_df = to_data_df(test, data_dir)
test_data_df.to_csv(f"others/{dataset}/test.csv",index_label="id")
val_data_df = to_data_df(val, data_dir)
val_data_df.to_csv(f"others/{dataset}/val.csv",index_label="id")

100%|█████████████████████████████████████████████████████████████████████████████| 1491/1491 [00:05<00:00, 296.48it/s]


# SANITY CHECK

In [38]:
data_df = test_data_df
import json

def reduce_by_alpha(text, rationale, fidelity_type="sufficiency"):
    reduced_text = ""
    # whitespace tokenization
    tokens = text.split()

    for idx in range(len(tokens)):
        try:
            if fidelity_type == "sufficiency" and rationale[idx] >= 0.5:
                reduced_text = reduced_text + tokens[idx] + " "
            elif fidelity_type == "comprehensiveness" and rationale[idx] < 0.5:
                reduced_text = reduced_text + tokens[idx] + " "
        except Exception as e:
            if fidelity_type == "comprehensiveness":
                reduced_text = reduced_text + tokens[idx] + " "

    # removed the last space from the text
    if len(reduced_text) > 0:
        reduced_text = reduced_text[:-1]

    return reduced_text

data_df = data_df[data_df['rationale'].notna()]
data_df.reset_index(drop=True, inplace=True)


data_df["sufficiency_text"] = data_df[
    ["text", "rationale"]].apply(lambda s: reduce_by_alpha(*s, fidelity_type="sufficiency"), axis=1)
data_df["comprehensiveness_text"] = data_df[
    ["text", "rationale"]].apply(lambda s: reduce_by_alpha(*s, fidelity_type="comprehensiveness"), axis=1)

In [39]:
data_df['sufficiency_text'][0]

"are black widow and red back spiders the same [SEP] Section::::Placement . A member of the genus Latrodectus in the family Theridiidae , the redback belongs in a clade with the black widow spider , with the katipo as its closest relative . A 2004 molecular study supports the redback 's status as a distinct species , as does the unique abdomen - presenting behaviour of the male during mating . The close relationship between the two species is shown when mating : the male redback is able to successfully mate with a female katipo producing hybrid offspring . However , the male katipo is too heavy to mate with the female redback , as it triggers a predatory response in the female when it approaches the web , causing the female to eat it . There is evidence of interbreeding between female katipo and male redbacks in the wild"

In [36]:
test["evidences"][0]

[[{'docid': 'BH_wiki_60_0',
   'end_sentence': 189,
   'end_token': 4179,
   'start_sentence': 181,
   'start_token': 3930,
   'text': 'Section::::Energy balance . All biomass goes through at least some of these steps : it needs to be grown , collected , dried , fermented , distilled , and burned . All of these steps require resources and an infrastructure . The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance ( or " energy returned on energy invested " ) . Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US : one unit of fossil - fuel energy is required to create 1.3 energy units from the resulting ethanol . The energy balance for sugarcane ethanol produced in Brazil is more favorable , with one unit of fossil - fuel energy required to create 8 from the ethanol . Energy balance estimates are not easily produced , th

In [41]:
test_data_df.iloc[0]

text              are black widow and red back spiders the same ...
classification                                                False
rationale         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
Name: 0, dtype: object